In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Groupby한 Series에서 4, 5 act_label 널값 처리를 못 하고 있다.

act = pd.read_csv('stackPerDevice.csv')
act.shape


(71957, 9)

In [18]:
act = act[ act['deviceId'] == 'id_MzQ6QUI6OTU6NzE6OUQ6NEM']
act

,deviceId,activity_key,avg_presence,avg_people_cnt,avg_activity,max_activity,cnt,first_eventtime,lastStatus
43582,id_MzQ6QUI6OTU6NzE6OUQ6NEM,2769930,0,0.0,9.260772,29.3991,172,NaN,NaN
43583,id_MzQ6QUI6OTU6NzE6OUQ6NEM,2769931,0,0.0,7.046147,34.1174,78,NaN,NaN
43584,id_MzQ6QUI6OTU6NzE6OUQ6NEM,2769932,0,0.0,6.565495,41.3503,87,NaN,NaN
43585,id_MzQ6QUI6OTU6NzE6OUQ6NEM,2769933,0,0.0,9.498293,36.1823,29,NaN,NaN
43586,id_MzQ6QUI6OTU6NzE6OUQ6NEM,2769934,0,0.0,5.249270,19.3683,27,NaN,NaN
...,...,...,...,...,...,...,...,...,...
45738,id_MzQ6QUI6OTU6NzE6OUQ6NEM,2774205,0,0.0,10.180770,51.7734,81,NaN,NaN
45739,id_MzQ6QUI6OTU6NzE6OUQ6NEM,2774206,0,0.0,10.595665,58.2283,23,NaN,NaN
45740,id_MzQ6QUI6OTU6NzE6OUQ6NEM,2774208,0,0.0,9.990581,41.0616,16,NaN,NaN
45741,id_MzQ6QUI6OTU6NzE6OUQ6NEM,2774210,0,0.0,0.105500,0.1861,2,NaN,NaN


In [19]:
act = act.drop(['deviceId'], axis=1)
act

,activity_key,avg_presence,avg_people_cnt,avg_activity,max_activity,cnt,first_eventtime,lastStatus
43582,2769930,0,0.0,9.260772,29.3991,172,NaN,NaN
43583,2769931,0,0.0,7.046147,34.1174,78,NaN,NaN
43584,2769932,0,0.0,6.565495,41.3503,87,NaN,NaN
43585,2769933,0,0.0,9.498293,36.1823,29,NaN,NaN
43586,2769934,0,0.0,5.249270,19.3683,27,NaN,NaN
...,...,...,...,...,...,...,...,...
45738,2774205,0,0.0,10.180770,51.7734,81,NaN,NaN
45739,2774206,0,0.0,10.595665,58.2283,23,NaN,NaN
45740,2774208,0,0.0,9.990581,41.0616,16,NaN,NaN
45741,2774210,0,0.0,0.105500,0.1861,2,NaN,NaN


In [20]:
ABSENCE = 0 # 부재
NOT_ACT = 1 # 미활동
ACT = 2 # 활동
CAUTION = 3 #주의
FALL = 4 # 낙상


def getEmergencyLabel(status, defaultStatus):
    if status == "fall_detected" or status == "fall_exit" or status == "fall_confirmed":
        return CAUTION
    elif status == "calling" or status == "finished":
        return FALL
    else:
        return defaultStatus

def makeStatus(df):
#     if df['avg_people_cnt'] == 0: #없는 경우가 있다.
#         return ABSENCE
#     else:
#     act_label = ABSENCE
    if df['avg_activity'] == 0:
        act_label = ABSENCE
    elif df['avg_activity'] < 3.0 or df['max_activity'] < 20.0:
        act_label = NOT_ACT
    else:
        act_label = ACT
        
    act_label = getEmergencyLabel(df['lastStatus'], act_label)
    return act_label

In [31]:
# #디바이스별로 집계를 구하고, 그 결과를 newdata에 저장함.
# newdata = pd.DataFrame()

# # group의 키 목록을 구함
# # grouped = act.groupby('deviceId')

# # for deviceid in grouped.groups.keys():
# #     newact = act[ act['deviceId'] == deviceid ].copy()
# newdata['act_label'] = newdata.apply(makeStatus, axis=1)
# #     newdata = newdata.append(newact)
    
# # newdata
# newdata

from datetime import datetime


def makeMD(x):
#     print(x['activity_key'])
    ts = datetime.fromtimestamp(x['activity_key']*600)
    return ts.strftime('%m-%d')

act['day'] = act.apply(makeMD, axis=1)
# act['day'] = act['activity_key'].apply(makeMD, axis=1)


act['act_label'] = act.apply(makeStatus, axis=1)

# deviceid, act_label별 집계수를 구한다.
act['new_cnt'] = act.groupby(['day','act_label'])['act_label'].transform('count')
act

,activity_key,avg_presence,avg_people_cnt,avg_activity,max_activity,cnt,first_eventtime,lastStatus,day,act_label,new_cnt
43582,2769930,0,0.0,9.260772,29.3991,172,NaN,NaN,09-01,2,82
43583,2769931,0,0.0,7.046147,34.1174,78,NaN,NaN,09-01,2,82
43584,2769932,0,0.0,6.565495,41.3503,87,NaN,NaN,09-01,2,82
43585,2769933,0,0.0,9.498293,36.1823,29,NaN,NaN,09-01,2,82
43586,2769934,0,0.0,5.249270,19.3683,27,NaN,NaN,09-01,1,19
...,...,...,...,...,...,...,...,...,...,...,...
45738,2774205,0,0.0,10.180770,51.7734,81,NaN,NaN,09-30,2,36
45739,2774206,0,0.0,10.595665,58.2283,23,NaN,NaN,09-30,2,36
45740,2774208,0,0.0,9.990581,41.0616,16,NaN,NaN,09-30,2,36
45741,2774210,0,0.0,0.105500,0.1861,2,NaN,NaN,09-30,1,15


In [32]:
newdata1 = act[['day', 'act_label', 'new_cnt']]
newdata1


,day,act_label,new_cnt
43582,09-01,2,82
43583,09-01,2,82
43584,09-01,2,82
43585,09-01,2,82
43586,09-01,1,19
...,...,...,...
45738,09-30,2,36
45739,09-30,2,36
45740,09-30,2,36
45741,09-30,1,15


In [33]:
newdata1 = newdata1.npan()
newdata1

,day,act_label,new_cnt
43582,09-01,2,82
43586,09-01,1,19
43623,09-01,3,3
43686,09-02,2,13
43690,09-02,1,3
...,...,...,...
45618,09-29,2,44
45621,09-29,1,29
45691,09-30,2,36
45692,09-30,1,15


In [34]:
gp = newdata1.groupby('day')

kim = pd.DataFrame(np.zeros((29,3)), columns=['day', 'act_label', 'act_cnt'])

index = 0 

for id in gp.groups.keys():
    kim.loc[index] = [id, 0, np.nan]
    index += 1
    kim.loc[index] = [id, 1, np.nan]
    index += 1
    kim.loc[index] = [id, 2, np.nan]
    index += 1
    kim.loc[index] = [id, 3, np.nan]
    index += 1
    kim.loc[index] = [id, 4, np.nan]
    index += 1

kim.act_cnt = np.nan
kim

,day,act_label,act_cnt
0,09-01,0.0,NaN
1,09-01,1.0,NaN
2,09-01,2.0,NaN
3,09-01,3.0,NaN
4,09-01,4.0,NaN
...,...,...,...
145,09-30,0.0,NaN
146,09-30,1.0,NaN
147,09-30,2.0,NaN
148,09-30,3.0,NaN


In [35]:
kim = pd.merge(kim, newdata1, on=['day', 'act_label'], how="left")
kim

,day,act_label,act_cnt,new_cnt
0,09-01,0.0,NaN,NaN
1,09-01,1.0,NaN,19.0
2,09-01,2.0,NaN,82.0
3,09-01,3.0,NaN,3.0
4,09-01,4.0,NaN,NaN
...,...,...,...,...
145,09-30,0.0,NaN,1.0
146,09-30,1.0,NaN,15.0
147,09-30,2.0,NaN,36.0
148,09-30,3.0,NaN,NaN


In [36]:
kim['new_cnt'] = kim['new_cnt'].fillna(0)
kim

,day,act_label,act_cnt,new_cnt
0,09-01,0.0,NaN,0.0
1,09-01,1.0,NaN,19.0
2,09-01,2.0,NaN,82.0
3,09-01,3.0,NaN,3.0
4,09-01,4.0,NaN,0.0
...,...,...,...,...
145,09-30,0.0,NaN,1.0
146,09-30,1.0,NaN,15.0
147,09-30,2.0,NaN,36.0
148,09-30,3.0,NaN,0.0


In [37]:
kim = kim.drop(['act_cnt'], axis=1)

In [38]:
reshaped = kim.pivot('day', 'act_label', 'new_cnt')
reshaped

C:\Users\sunin\AppData\Local\Temp\ipykernel_42732\1684931186.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  reshaped = kim.pivot('day', 'act_label', 'new_cnt')


act_label,0.0,1.0,2.0,3.0,4.0
day,,,,,
09-01,0.0,19.0,82.0,3.0,0.0
09-02,0.0,3.0,13.0,0.0,0.0
09-03,0.0,39.0,64.0,0.0,0.0
09-04,0.0,22.0,100.0,3.0,0.0
09-05,0.0,19.0,94.0,11.0,1.0
09-06,0.0,18.0,80.0,10.0,0.0
09-07,0.0,13.0,68.0,4.0,0.0
09-08,1.0,28.0,54.0,0.0,0.0
09-09,1.0,28.0,72.0,0.0,0.0


In [70]:
reshaped = reshaped.reset_index()

In [71]:
reshaped

day new_cnt                           
act_label           0.0    1.0     2.0    3.0  4.0
0          0901     0.0  655.0  1141.0  438.0  0.0
1          0902     0.0  655.0  1141.0  438.0  0.0
2          0903     0.0  655.0  1141.0  438.0  0.0
3          0904     0.0  655.0  1141.0  438.0  0.0
4          0905     0.0  655.0  1141.0  438.0  0.0
5          0906    19.0  655.0  1141.0  438.0  0.0
6          0907     0.0  655.0  1141.0  438.0  0.0
7          0908     0.0  655.0  1141.0  438.0  0.0
8          0909     0.0  655.0  1141.0  438.0  0.0
9          0910     0.0  655.0     0.0    0.0  0.0
10         0911    19.0  655.0  1141.0  438.0  0.0
11         0912     0.0  655.0  1141.0  438.0  0.0
12         0913     0.0  655.0  1141.0  438.0  0.0
13         0914    19.0  655.0  1141.0  438.0  6.0
14         0915     0.0  655.0  1141.0  438.0  0.0
15         0916     0.0  655.0  1141.0    0.0  0.0
16         0917     0.0  655.0  1141.0  438.0  0.0
17         0918    19.0  655.0  1141.0  438.0  0.0
18         0919     0.0  655.0  1141.0  438.0  0.0
19         0920     0.0  655.0     0.0  438.0  0.0
20         0921    19.0  655.0  1141.0  438.0  0.0
21         0922    19.0  655.0  1141.0  438.0  0.0
22         0923     0.0  655.0  1141.0  438.0  6.0
23         0924    19.0  655.0  1141.0  438.0  0.0
24         0925    19.0  655.0  1141.0  438.0  6.0
25         0926    19.0  655.0  1141.0  438.0  0.0
26         0927     0.0  655.0  1141.0  438.0  6.0
27         0928    19.0  655.0  1141.0  438.0  0.0
28         0929     0.0  655.0  1141.0  438.0  0.0
29         0930    19.0  655.0  1141.0  438.0  6.0

In [117]:
reshaped.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30 entries, 0901 to 0930
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0.0     30 non-null     float64
 1   1.0     30 non-null     float64
 2   2.0     30 non-null     float64
 3   3.0     30 non-null     float64
 4   4.0     30 non-null     float64
dtypes: float64(5)
memory usage: 1.4+ KB


In [39]:
reshaped.to_csv('stacked_one_19.csv')